In [20]:
import pandas as pd
import openpyxl
import matplotlib.pyplot as plt
import plotly.express as px


In [21]:
bolsa = pd.read_excel("C:/Users/evasquez/OneDrive - Bolsa de Cereales de Buenos Aires/Documentos - Instituto Estudios Económicos/Pedidos/2024/Diferencias_FOB/Input/serie_fob.xlsx")
bolsa = bolsa.dropna(subset=["Precio"]).rename(columns={"Precio": "FOB_Bolsa"})
bolsa["Fecha"] = pd.to_datetime(bolsa["Fecha"], format="%d/%m/%Y")
bolsa["Cereal"].replace("Trigo 11.5%", "Trigo", inplace=True)


# Leer el archivo Excel "serie_precios_minagro.xlsx"
minagro = pd.read_excel("C:/Users/evasquez/OneDrive - Bolsa de Cereales de Buenos Aires/Documentos - Instituto Estudios Económicos/Pedidos/2024/Diferencias_FOB/Input/serie_precios_minagro.xlsx")
minagro = minagro.dropna(subset=["Precio"]).query("Precio != 0").rename(columns={"Precio": "FOB_Minagro"})
minagro["Fecha"] = pd.to_datetime(minagro["Fecha"], format="%d/%m/%Y")
minagro["Cereal"].replace("Trigo Pan", "Trigo", inplace=True)

In [22]:
# Combinar los DataFrames 'bolsa' y 'minagro'
todo = pd.merge(bolsa, minagro)

# Crear una nueva columna 'posicion'
todo["posicion"] = pd.to_datetime(todo["Año"].astype(str) + '-' + todo["Mes"].astype(str) + '-01')

#Crear una columna con las diferencia de meses 
todo['Fecha'] = pd.to_datetime(todo['Fecha'])
todo['posicion'] = pd.to_datetime(todo['posicion'])

todo['time_delta_months'] = todo['posicion'].dt.to_period('M').view(dtype='int64') - todo['Fecha'].dt.to_period('M').view(dtype='int64')

todo['Embarque'] =  't+' + todo['time_delta_months'].astype(str)


In [23]:
#quiero que me muestre precios de trigo de mes 5 y año 2025
todo = todo.query("Cereal == 'Trigo' & Año == 2025 & Mes == 1")


In [24]:
#todo.to_excel("C:/Users/evasquez/Downloads/todo.xlsx", index=False)

In [25]:
# Cambiar el formato de la columna 'Fecha' a texto en el DataFrame 'todo'
todo['Fecha'] = todo['Fecha'].dt.strftime('%d/%m/%Y')

In [26]:
#quiero que grafiques los precios de FOB de bolsa y minagro en el tiempo
fig = px.line(todo, x="Fecha", y=["FOB_Bolsa", "FOB_Minagro"], title="Precios de FOB de Bolsa y Minagro a Cosecha")
fig.show()

c:\Users\evasquez\AppData\Local\Programs\Python\Python312\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [27]:
todo['MEES'] = todo['Fecha'].dt.month

# Calcula el promedio de los precios para cada mes
promedio = todo.groupby('MEES')[['FOB_Bolsa', 'FOB_Minagro']].mean()

# Calcula la diferencia de precios promedio
promedio['Diferencia'] = promedio['FOB_Minagro'] - promedio['FOB_Bolsa']
promedio

,FOB_Bolsa,FOB_Minagro,Diferencia
MEES,,,
2,223.142857,230.285714,7.142857
3,214.666667,223.944444,9.277778
4,232.500000,239.526316,7.026316
5,265.952381,271.666667,5.714286
6,264.233333,271.000000,6.766667
7,240.510000,247.650000,7.140000
8,229.668182,238.000000,8.331818
9,227.176471,234.000000,6.823529
